In [33]:
import lightgbm as lgb
import pandas as pd
import pickle
import numpy as np
import csv

In [22]:
# project_root_path = '/content/drive/MyDrive/NCCU1102/WSM/proj3/RecSys-Dressipi/'
project_root_path = '../..'

In [23]:
%%time
from sklearn.model_selection import train_test_split

# fit training data
X_path = project_root_path + '/dataset/train_features/train_X_0.pickle'
y_path = project_root_path + '/dataset/train_features/train_y_0.pickle'
with open( X_path, 'rb') as f:
    X = pickle.load(f)
with open( y_path, 'rb') as f:
    y = pickle.load(f)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)


CPU times: user 8.34 s, sys: 19.9 s, total: 28.3 s
Wall time: 40.5 s


In [9]:
len(X_test)

300000

In [24]:
# query_train = [len(X_train)]
query_train = [150] * (len(X_train)//150)
query_val = [150] * (len(X_val)//150) 
query_test = [150] * (len(X_test)//150) 

In [25]:

gbm = lgb.LGBMRanker()


gbm.fit(X_train, y_train, group=query_train)


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRanker()

In [28]:
def pred2rank_result( session_id, X, pred ):
    top100_index = np.argsort(pred)[-100:]
    rank_result = []
    for count, index in enumerate(top100_id[::-1]):
        row = ','.join([str(session_id), str(int(X[index][0])), str(count+1)])
        rank_result.append(row)
    return rank_result

def predict_session( gbm, session_id ):
    with open('../../dataset/test_features/test_X_' + str(session_id) + '.pickle', 'rb') as f:
        X = pickle.load(f)
    pred = gbm.predict(X)
    return pred2rank_result( session_id, X, pred )



In [31]:
with open('../../dataset/test_leaderboard_uniq_sessions') as f:
    sessions = [ int(line) for line in f.readlines()[1:]]

results = []
for session_id in sessions:
    results.extend(predict_session(gbm, session_id ))

In [39]:
with open('../../result/lgbm_train0.csv', 'w') as f:
    f.write('session_id,item_id,rank\n')
    f.write('\n'.join(results))